In [6]:
# -*- coding: utf-8 -*-
# ipynb 공통 유틸: 경로, split 생성, config 안전 패치

import json
import random
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import pandas as pd

from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.utils import register_all_modules


# -----------------------
# 1) 고정 경로 (네 환경 기준)
# -----------------------

# mmdetection 루트 (configs 탐색용)
MMD_ROOT = Path("/data/ephemeral/home/model/baseline/mmdetection")

# 데이터 루트
FULL_DATA_ROOT = Path("/data/ephemeral/home/model/dataset")
TRAIN_JSON_FULL = FULL_DATA_ROOT / "train.json"
TEST_JSON_FULL  = FULL_DATA_ROOT / "test.json"

# 샘플 제출 폴더
SAMPLE_SUB_DIR = Path("/data/ephemeral/home/model/sample_submission")

# 단일 모델 실험 산출물 저장 위치
WORK_DIR_ROOT = Path("/data/ephemeral/home/model/baseline/ensemble copy/work_dirs")

WORK_DIR_ROOT.mkdir(parents=True, exist_ok=True)

# 이미지 스케일 고정
IMAGE_SCALE = (1024, 1024)

# 클래스 10개 (대회 공지 기준 순서)
CLASSES = (
    "General trash",
    "Paper",
    "Paper pack",
    "Metal",
    "Glass",
    "Plastic",
    "Styrofoam",
    "Plastic bag",
    "Battery",
    "Clothing",
)

# split 설정
RANDOM_SEED = 42
TRAIN_RATIO = 0.9   # 0.8/0.2보다 학습 데이터를 늘려 단일 모델 성능 안정성을 노림


print("MMD_ROOT:", MMD_ROOT)
print("FULL_DATA_ROOT:", FULL_DATA_ROOT)
print("TRAIN_JSON_FULL:", TRAIN_JSON_FULL)
print("TEST_JSON_FULL :", TEST_JSON_FULL)
print("SAMPLE_SUB_DIR :", SAMPLE_SUB_DIR)
print("WORK_DIR_ROOT  :", WORK_DIR_ROOT)


# -----------------------
# 2) COCO json 로드/저장
# -----------------------

def load_coco(json_path: Path) -> Dict:
    with open(json_path, "r") as f:
        return json.load(f)

def save_coco(data: Dict, json_path: Path) -> None:
    json_path.parent.mkdir(parents=True, exist_ok=True)
    with open(json_path, "w") as f:
        json.dump(data, f)


# -----------------------
# 3) train/val split 생성
# -----------------------

def make_train_val_split(
    src_json: Path,
    out_dir: Path,
    train_ratio: float = 0.9,
    seed: int = 42
) -> Dict[str, Path]:
    """
    이미지 id 기준 랜덤 분할.
    detection에서 가장 기본적이고 안전한 방식.
    """
    random.seed(seed)

    data = load_coco(src_json)
    images = data["images"]
    anns = data["annotations"]

    img_ids = [img["id"] for img in images]
    random.shuffle(img_ids)

    split_idx = int(len(img_ids) * train_ratio)
    train_ids = set(img_ids[:split_idx])
    val_ids   = set(img_ids[split_idx:])

    def _filter(ids: set):
        # ids에 해당하는 image만 남김
        imgs = [img for img in images if img["id"] in ids]
        img_set = {img["id"] for img in imgs}
        # 해당 이미지에 매칭되는 annotation만 남김
        filtered_anns = [ann for ann in anns if ann["image_id"] in img_set]
        return {**data, "images": imgs, "annotations": filtered_anns}

    out_dir.mkdir(parents=True, exist_ok=True)
    train_json = out_dir / "train_split.json"
    val_json   = out_dir / "val_split.json"

    save_coco(_filter(train_ids), train_json)
    save_coco(_filter(val_ids), val_json)

    return {"train": train_json, "val": val_json}


# -----------------------
# 4) pipeline 스케일 고정
# -----------------------

def set_img_scale(pipeline, scale):
    """
    Resize / RandomResize / RandomChoiceResize 등을 1024로 통일.
    """
    for t in pipeline:
        if isinstance(t, list):
            set_img_scale(t, scale)
            continue
        if not isinstance(t, dict):
            continue

        if t.get("type") in ("Resize", "RandomResize", "RandomChoiceResize"):
            if "scale" in t:
                t["scale"] = scale
            if "img_scale" in t:
                t["img_scale"] = scale
            if "scales" in t:
                t["scales"] = [scale]

        if "transforms" in t:
            set_img_scale(t["transforms"], scale)


# -----------------------
# 5) num_classes 재귀 고정
# -----------------------

def set_num_classes(model_cfg, num_classes: int):
    """
    다양한 head 구조에서 num_classes를 재귀적으로 10으로 맞춤.
    """
    if isinstance(model_cfg, dict):
        if "num_classes" in model_cfg:
            model_cfg["num_classes"] = num_classes
        for v in model_cfg.values():
            set_num_classes(v, num_classes)
    elif isinstance(model_cfg, list):
        for v in model_cfg:
            set_num_classes(v, num_classes)


# -----------------------
# 6) 샘플 csv 자동 선택
# -----------------------

def pick_sample_csv(sample_dir: Path) -> Path:
    """
    - 이름에 mmdetection 포함된 샘플을 우선
    - 없으면 첫 번째 csv
    """
    csvs = sorted(sample_dir.glob("*.csv"))
    if not csvs:
        raise FileNotFoundError(f"샘플 제출 csv가 없습니다: {sample_dir}")

    for c in csvs:
        if "mmdetection" in c.name.lower():
            return c

    return csvs[0]


# -----------------------
# 7) base config 자동 탐색
# -----------------------

def pick_best_single_base_cfg() -> Path:
    """
    앙상블에 없던 단일 후보로 RTMDet 계열을 우선 시도.
    환경에 따라 config 파일명이 다를 수 있어
    여러 후보 중 '존재하는 첫 번째'를 선택.
    """
    candidates = [
        "configs/rtmdet/rtmdet_l_8xb32-300e_coco.py",
        "configs/rtmdet/rtmdet_m_8xb32-300e_coco.py",
        "configs/rtmdet/rtmdet_s_8xb32-300e_coco.py",
        # 혹시 RTMDet가 없다면 2순위 후보
        "configs/yolox/yolox_l_8xb8-300e_coco.py",
        "configs/yolox/yolox_m_8xb8-300e_coco.py",
    ]

    for rel in candidates:
        p = MMD_ROOT / rel
        if p.exists():
            print("Selected base cfg:", p)
            return p

    # 마지막 안전장치: 그래도 없으면 에러로 명확히 알려줌
    raise FileNotFoundError(
        "RTMDet/YOLOX 후보 config를 찾지 못했습니다. "
        "MMD_ROOT/configs 아래 실제 파일명을 확인해 주세요."
    )


MMD_ROOT: /data/ephemeral/home/model/baseline/mmdetection
FULL_DATA_ROOT: /data/ephemeral/home/model/dataset
TRAIN_JSON_FULL: /data/ephemeral/home/model/dataset/train.json
TEST_JSON_FULL : /data/ephemeral/home/model/dataset/test.json
SAMPLE_SUB_DIR : /data/ephemeral/home/model/sample_submission
WORK_DIR_ROOT  : /data/ephemeral/home/model/baseline/ensemble copy/work_dirs


In [8]:
# -*- coding: utf-8 -*-
# RTMDet 단일 모델 전체 테스트 추론 (ipynb 단독 실행 안전 버전)
# - test.json 전체 대상으로 COCO bbox json 생성
# - 샘플 템플릿 고정 방식으로 submission 생성

from pathlib import Path
from typing import Optional, Dict
import json
import pandas as pd

from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.utils import register_all_modules


# -----------------------
# 0) 경로/상수 (학습 로그 기준으로 정확히 맞춤)
# -----------------------

MMD_ROOT = Path("/data/ephemeral/home/model/baseline/mmdetection")

FULL_DATA_ROOT = Path("/data/ephemeral/home/model/dataset")
TEST_JSON_FULL  = FULL_DATA_ROOT / "test.json"

SAMPLE_SUB_DIR  = Path("/data/ephemeral/home/model/sample_submission")

# 중요: 네 학습 로그의 Final work_dir 기준
WORK_DIR_ROOT = Path("/data/ephemeral/home/model/work_dirs_single")
EXP_NAME = "rtmdet_1024_single"

EXP_WORK_DIR = WORK_DIR_ROOT / EXP_NAME

IMAGE_SCALE = (1024, 1024)

CLASSES = (
    "General trash",
    "Paper",
    "Paper pack",
    "Metal",
    "Glass",
    "Plastic",
    "Styrofoam",
    "Plastic bag",
    "Battery",
    "Clothing",
)


# -----------------------
# 1) 유틸 함수들
# -----------------------

def pick_sample_csv(sample_dir: Path) -> Path:
    csvs = sorted(sample_dir.glob("*.csv"))
    if not csvs:
        raise FileNotFoundError(f"샘플 제출 csv 없음: {sample_dir}")
    for c in csvs:
        if "mmdetection" in c.name.lower():
            return c
    return csvs[0]


def set_img_scale(pipeline, scale):
    for t in pipeline:
        if isinstance(t, list):
            set_img_scale(t, scale)
            continue
        if not isinstance(t, dict):
            continue

        if t.get("type") in ("Resize", "RandomResize", "RandomChoiceResize"):
            if "scale" in t:
                t["scale"] = scale
            if "img_scale" in t:
                t["img_scale"] = scale
            if "scales" in t:
                t["scales"] = [scale]

        if "transforms" in t:
            set_img_scale(t["transforms"], scale)


def set_num_classes(model_cfg, num_classes: int):
    if isinstance(model_cfg, dict):
        if "num_classes" in model_cfg:
            model_cfg["num_classes"] = num_classes
        for v in model_cfg.values():
            set_num_classes(v, num_classes)
    elif isinstance(model_cfg, list):
        for v in model_cfg:
            set_num_classes(v, num_classes)


def find_checkpoint(work_dir: Path) -> Optional[Path]:
    # 1) last_checkpoint 우선
    last_txt = work_dir / "last_checkpoint"
    if last_txt.exists():
        p = last_txt.read_text().strip()
        if p:
            ck = Path(p)
            # 상대경로일 수 있어 보정
            if not ck.is_absolute():
                ck = work_dir / ck
            if ck.exists():
                return ck

    # 2) 없으면 최신 *.pth
    ckpts = sorted(work_dir.glob("*.pth"))
    if ckpts:
        return ckpts[-1]

    return None


def parse_test_images(test_json: Path) -> Dict[int, str]:
    with open(test_json, "r") as f:
        data = json.load(f)
    return {img["id"]: img["file_name"] for img in data["images"]}


def coco_bbox_json_to_prediction_dict(
    bbox_json: Path,
    test_json: Path,
    score_thr: float = 0.05
) -> Dict[str, str]:

    id_to_fname = parse_test_images(test_json)

    with open(bbox_json, "r") as f:
        dets = json.load(f)

    per_image = {}
    for d in dets:
        score = float(d.get("score", 0.0))
        if score < score_thr:
            continue

        img_id = int(d["image_id"])
        cat_id = int(d["category_id"])

        x, y, w, h = d["bbox"]
        x1, y1, x2, y2 = x, y, x + w, y + h

        per_image.setdefault(img_id, []).append((cat_id, score, x1, y1, x2, y2))

    pred_dict = {}
    for img_id, fname in id_to_fname.items():
        preds = sorted(per_image.get(img_id, []), key=lambda x: x[1], reverse=True)

        tokens = []
        for cat, sc, x1, y1, x2, y2 in preds:
            tokens.extend([
                str(cat),
                f"{sc:.6f}",
                f"{x1:.1f}",
                f"{y1:.1f}",
                f"{x2:.1f}",
                f"{y2:.1f}",
            ])

        pred_dict[fname] = " ".join(tokens)

    return pred_dict


def save_submission_with_sample(
    pred_dict: Dict[str, str],
    sample_csv_path: Path,
    out_csv_path: Path
) -> None:

    sample = pd.read_csv(sample_csv_path)

    if "image_id" not in sample.columns or "PredictionString" not in sample.columns:
        raise ValueError(f"샘플 컬럼이 예상과 다름: {list(sample.columns)}")

    sample["PredictionString"] = sample["image_id"].map(lambda x: pred_dict.get(x, ""))

    out_csv_path.parent.mkdir(parents=True, exist_ok=True)
    sample.to_csv(out_csv_path, index=False)

    print("Saved submission:", out_csv_path)


# -----------------------
# 2) 사전 체크
# -----------------------

assert TEST_JSON_FULL.exists(), f"test.json 없음: {TEST_JSON_FULL}"
assert SAMPLE_SUB_DIR.exists(), f"sample_submission 폴더 없음: {SAMPLE_SUB_DIR}"
assert EXP_WORK_DIR.exists(), f"실험 폴더 없음: {EXP_WORK_DIR}"

sample_csv = pick_sample_csv(SAMPLE_SUB_DIR)
print("Using sample csv:", sample_csv)

ckpt = find_checkpoint(EXP_WORK_DIR)
if ckpt is None:
    raise FileNotFoundError(f"checkpoint 없음: {EXP_WORK_DIR}")

print("checkpoint:", ckpt)


# -----------------------
# 3) base cfg 지정
#    학습 로그에서 선택된 cfg와 동일하게 맞추는 것이 가장 안정적
# -----------------------

base_cfg_path = MMD_ROOT / "configs/rtmdet/rtmdet_l_8xb32-300e_coco.py"
assert base_cfg_path.exists(), f"base cfg 없음: {base_cfg_path}"
print("Using base cfg:", base_cfg_path)

register_all_modules(init_default_scope=True)

test_cfg = Config.fromfile(str(base_cfg_path))
test_cfg.default_scope = "mmdet"

# data_root
test_cfg.data_root = str(FULL_DATA_ROOT)

# test_dataloader 구성
if "test_dataloader" not in test_cfg:
    test_cfg.test_dataloader = test_cfg.val_dataloader

loader = test_cfg.test_dataloader
ds_cfg = loader["dataset"] if "dataset" in loader else loader

ds_cfg.metainfo = dict(classes=CLASSES)
ds_cfg.data_root = str(FULL_DATA_ROOT)
ds_cfg.ann_file = str(TEST_JSON_FULL)
ds_cfg.data_prefix = dict(img="")

# pipeline 스케일 고정
if hasattr(ds_cfg, "pipeline"):
    set_img_scale(ds_cfg.pipeline, IMAGE_SCALE)

# 클래스 수 고정
set_num_classes(test_cfg.model, len(CLASSES))

# 배치/워커
test_cfg.test_dataloader.batch_size = 1
test_cfg.test_dataloader.num_workers = 2

# evaluator: COCO bbox json만 저장
out_prefix = EXP_WORK_DIR / f"{EXP_NAME}_test"
test_cfg.test_evaluator = dict(
    type="CocoMetric",
    ann_file=str(TEST_JSON_FULL),
    metric="bbox",
    format_only=True,
    outfile_prefix=str(out_prefix),
)

# checkpoint 로드
test_cfg.load_from = str(ckpt)

# 학습 관련 설정 제거
test_cfg.train_dataloader = None
test_cfg.train_cfg = None
test_cfg.optim_wrapper = None
if hasattr(test_cfg, "param_scheduler"):
    test_cfg.param_scheduler = None
if hasattr(test_cfg, "val_dataloader"):
    test_cfg.val_dataloader = None
if hasattr(test_cfg, "val_cfg"):
    test_cfg.val_cfg = None
if hasattr(test_cfg, "val_evaluator"):
    test_cfg.val_evaluator = None

# work_dir 고정
test_cfg.work_dir = str(EXP_WORK_DIR)


# -----------------------
# 4) Runner test 실행
# -----------------------

runner = Runner.from_cfg(test_cfg)
runner.test()


# -----------------------
# 5) bbox json -> submission
# -----------------------

bbox_json = EXP_WORK_DIR / f"{EXP_NAME}_test.bbox.json"
if not bbox_json.exists():
    cand = sorted(EXP_WORK_DIR.glob("*_test.bbox.json"))
    if cand:
        bbox_json = cand[-1]

print("bbox json:", bbox_json)
if not bbox_json.exists():
    raise FileNotFoundError("bbox json 생성 실패")

pred_dict = coco_bbox_json_to_prediction_dict(
    bbox_json=bbox_json,
    test_json=TEST_JSON_FULL,
    score_thr=0.05
)

out_csv = EXP_WORK_DIR / f"{EXP_NAME}_submission_full.csv"
save_submission_with_sample(pred_dict, sample_csv, out_csv)

print("단일 모델 전체 테스트 추론 완료")
print("제출 파일:", out_csv)


Using sample csv: /data/ephemeral/home/model/sample_submission/faster_rcnn_mmdetection_submission.csv
checkpoint: /data/ephemeral/home/model/work_dirs_single/rtmdet_1024_single/epoch_12.pth
Using base cfg: /data/ephemeral/home/model/baseline/mmdetection/configs/rtmdet/rtmdet_l_8xb32-300e_coco.py
12/07 16:20:43 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 47184910
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: n/a
    PyTorch: 2.1.2+cu121
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
 

/bin/sh: 1: gcc: not found


12/07 16:20:44 - mmengine - INFO - Config:
auto_scale_lr = dict(base_batch_size=16, enable=False)
backend_args = None
base_lr = 0.004
custom_hooks = [
    dict(
        ema_type='ExpMomentumEMA',
        momentum=0.0002,
        priority=49,
        type='EMAHook',
        update_buffers=True),
    dict(
        switch_epoch=280,
        switch_pipeline=[
            dict(backend_args=None, type='LoadImageFromFile'),
            dict(type='LoadAnnotations', with_bbox=True),
            dict(
                keep_ratio=True,
                ratio_range=(
                    0.1,
                    2.0,
                ),
                scale=(
                    640,
                    640,
                ),
                type='RandomResize'),
            dict(crop_size=(
                640,
                640,
            ), type='RandomCrop'),
            dict(type='YOLOXHSVRandomAug'),
            dict(prob=0.5, type='RandomFlip'),
            dict(
                pad_val=d

/data/ephemeral/home/py310/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


12/07 16:20:58 - mmengine - INFO - Epoch(test) [  50/4871]    eta: 0:11:14  time: 0.1400  data_time: 0.0066  memory: 734  
12/07 16:21:03 - mmengine - INFO - Epoch(test) [ 100/4871]    eta: 0:09:29  time: 0.0990  data_time: 0.0025  memory: 734  
12/07 16:21:08 - mmengine - INFO - Epoch(test) [ 150/4871]    eta: 0:08:53  time: 0.1001  data_time: 0.0025  memory: 734  
12/07 16:21:13 - mmengine - INFO - Epoch(test) [ 200/4871]    eta: 0:08:30  time: 0.0985  data_time: 0.0025  memory: 734  
12/07 16:21:18 - mmengine - INFO - Epoch(test) [ 250/4871]    eta: 0:08:16  time: 0.0998  data_time: 0.0027  memory: 734  
12/07 16:21:23 - mmengine - INFO - Epoch(test) [ 300/4871]    eta: 0:08:05  time: 0.1001  data_time: 0.0026  memory: 734  
12/07 16:21:28 - mmengine - INFO - Epoch(test) [ 350/4871]    eta: 0:07:56  time: 0.1005  data_time: 0.0030  memory: 734  
12/07 16:21:32 - mmengine - INFO - Epoch(test) [ 400/4871]    eta: 0:07:44  time: 0.0929  data_time: 0.0025  memory: 734  
12/07 16:21:35 -